**22. Top 5 de lenguajes que son usados por usuarios bilingües. (⭐⭐).**

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=979a2f86623d395307b8893adcb2ac781832349c7e3f8a0c348caa7cb4e34166
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,008 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal In

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [5]:
sqlContext = SQLContext(sc)
df_languages = sqlContext.read.csv('/content/drive/MyDrive/datosTP1/languages.csv', header=True, inferSchema=True)
rdd_languages = df_languages.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
rdd_languages.take(5)

[Row(babel_user=4502458, babel_lang='FR', babel_level='2'),
 Row(babel_user=5928200, babel_lang='aa', babel_level='0'),
 Row(babel_user=46918, babel_lang='ab', babel_level='0'),
 Row(babel_user=2050449, babel_lang='ab', babel_level='1'),
 Row(babel_user=4715583, babel_lang='ace', babel_level='0')]

Considero que un usuario sabe un idioma cuando su nivel es mayor a 1.

In [7]:
lenguajes_hablados = rdd_languages.filter(lambda x: x[2] != "0" and x[2] != "1")

Cuento cuantos idiomas habla cada usuario y me lo guardo en un diccionario.

In [8]:
cantidad_idiomas_usuarios = lenguajes_hablados.map(lambda x: (x[0], 1)).countByKey()

Filtro para quedarme con los usuarios que hablan más de un idioma, luego cuento y obtengo lo pedido.

In [9]:
lenguajes_hablados.filter(lambda x: cantidad_idiomas_usuarios[x[0]] > 1).map(lambda x: (x[1].lower(), 1)).reduceByKey(lambda x,y: x+y).takeOrdered(5, key=lambda x: -x[1])

[('en', 8429), ('es', 8299), ('fr', 1995), ('ca', 1025), ('de', 907)]